In [2]:
import torch

class TinyModel(torch.nn.Module):

    def __init__(self):
        super(TinyModel, self).__init__()

        self.linear1 = torch.nn.Linear(100, 200)
        self.activation = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(200, 10)
        self.softmax = torch.nn.Softmax()

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x

tinymodel = TinyModel()

print('The model:')
print(tinymodel)

print('\n\nJust one layer:')
print(tinymodel.linear2)

print('\n\nModel params:')
for param in tinymodel.parameters():
    print(param)

print('\n\nLayer params:')
for param in tinymodel.linear2.parameters():
    print(param)

The model:
TinyModel(
  (linear1): Linear(in_features=100, out_features=200, bias=True)
  (activation): ReLU()
  (linear2): Linear(in_features=200, out_features=10, bias=True)
  (softmax): Softmax(dim=None)
)


Just one layer:
Linear(in_features=200, out_features=10, bias=True)


Model params:
Parameter containing:
tensor([[-0.0401,  0.0542, -0.0618,  ...,  0.0447,  0.0945, -0.0045],
        [-0.0169,  0.0734, -0.0650,  ..., -0.0765,  0.0857,  0.0466],
        [-0.0149, -0.0215, -0.0653,  ..., -0.0998,  0.0877,  0.0573],
        ...,
        [ 0.0544,  0.0401, -0.0527,  ..., -0.0015,  0.0106,  0.0406],
        [-0.0898,  0.0129,  0.0853,  ..., -0.0466,  0.0042, -0.0108],
        [-0.0132, -0.0213, -0.0277,  ..., -0.0909,  0.0525, -0.0114]],
       requires_grad=True)
Parameter containing:
tensor([-0.0038,  0.0304, -0.0014,  0.0410,  0.0029,  0.0830,  0.0047,  0.0546,
        -0.0904,  0.0124, -0.0101, -0.0458, -0.0225,  0.0919, -0.0443, -0.0590,
         0.0711,  0.0415, -0.0243,  0.04

In [5]:
# Linear layer

lin = torch.nn.Linear(3, 2)
x = torch.rand(1, 3)
print('Input:')
print(x)

print('\n\nWeight and Bias parameters:')
for param in lin.parameters():
    print(param)

y = lin(x)
print('\n\nOutput:')
print(y)

Input:
tensor([[0.2714, 0.5355, 0.0601]])


Weight and Bias parameters:
Parameter containing:
tensor([[-0.5156,  0.4255, -0.4125],
        [-0.0899, -0.3691,  0.0729]], requires_grad=True)
Parameter containing:
tensor([-0.4603, -0.4347], requires_grad=True)


Output:
tensor([[-0.3972, -0.6525]], grad_fn=<AddmmBackward0>)


In [6]:
import torch.functional as F


class LeNet(torch.nn.Module):

    def __init__(self):
        super(LeNet, self).__init__()
        # 1 input image channel (black & white), 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = torch.nn.Conv2d(1, 6, 5)
        self.conv2 = torch.nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = torch.nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc3 = torch.nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [7]:
# RNN

class LSTMTagger(torch.nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = torch.nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = torch.nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = torch.nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [4]:
# Other non-learning layers
# Max pooling

my_tensor = torch.rand(1, 6, 6)
print(my_tensor)

maxpool_layer = torch.nn.MaxPool2d(3)
print(maxpool_layer(my_tensor)) 

tensor([[[0.0704, 0.0677, 0.4209, 0.9724, 0.9198, 0.7782],
         [0.4635, 0.6394, 0.6265, 0.9837, 0.7898, 0.7967],
         [0.5524, 0.5817, 0.5782, 0.9593, 0.2666, 0.1690],
         [0.2643, 0.2477, 0.1438, 0.7373, 0.6290, 0.7404],
         [0.8417, 0.9429, 0.5725, 0.6315, 0.3002, 0.2141],
         [0.5314, 0.0663, 0.8556, 0.4556, 0.6065, 0.3402]]])
tensor([[[0.6394, 0.9837],
         [0.9429, 0.7404]]])


In [5]:
# Normalization layers

my_tensor = torch.rand(1, 4, 4) * 20 + 5
print(my_tensor)

print(my_tensor.mean())

norm_layer = torch.nn.BatchNorm1d(4)
normed_tensor = norm_layer(my_tensor)
print(normed_tensor)

print(normed_tensor.mean())

tensor([[[22.8049, 18.2147,  9.3425,  7.0408],
         [12.6681, 14.5861, 23.6404,  6.0753],
         [14.8111, 20.0515, 15.3228, 11.3324],
         [ 5.1860, 23.2855,  8.1309, 19.3826]]])
tensor(14.4922)
tensor([[[ 1.3166,  0.6018, -0.7800, -1.1384],
         [-0.2508,  0.0547,  1.4971, -1.3010],
         [-0.1831,  1.5053, -0.0182, -1.3039],
         [-1.1687,  1.2322, -0.7780,  0.7145]]],
       grad_fn=<NativeBatchNormBackward0>)
tensor(-2.9802e-08, grad_fn=<MeanBackward0>)


In [6]:
# Dropout layer

my_tensor = torch.rand(1, 4, 4)

dropout = torch.nn.Dropout(p=0.4)
print(dropout(my_tensor))
print(dropout(my_tensor))

tensor([[[0.8480, 1.1984, 0.0000, 0.7435],
         [0.0000, 0.1335, 0.0000, 0.5203],
         [0.4302, 0.0000, 1.2287, 0.4959],
         [1.6579, 0.0000, 0.0000, 0.0000]]])
tensor([[[0.8480, 1.1984, 0.0000, 0.7435],
         [0.0000, 0.1335, 1.5447, 0.5203],
         [0.0000, 0.0145, 0.0000, 0.4959],
         [0.0000, 1.1870, 0.6534, 0.0000]]])


In [ ]:
# torch.nn.Module has many major activation functions and loss functions